In [2]:
import os
import xarray as xr
from dask.distributed import Client
import matplotlib.pyplot as plt
from os import listdir
import rioxarray as rxr
import glob

from CPF import *
from grizzly import *
from coagmet import *
from disdrometer import *
from mrms_gage import *
from usgs_other import *
from elevation import *
from usgs_new import *

# Create a path to the code file
codeDir = os.path.dirname(os.path.abspath(os.getcwd()))
parentDir = os.path.dirname(codeDir)

gage_folder = os.path.join(parentDir,"precip_gage")

# grizzly
grizzly = get_grizzly(gage_folder)
# cpf
cpf = get_cpf(gage_folder+'\\cpf google drive')
# disdrometer
disdrom = get_disdrom(gage_folder)
# coagmet
coag = get_coagmet(gage_folder)
# other gages
other = get_usgs_other(gage_folder)

new_usgs = get_usgsnew(gage_folder)

# bring everything together
gage = {**grizzly, **cpf, **disdrom, **coag, **other, **new_usgs}

# get list of keys (lat/lon)
coord = [i for i in gage.keys()]

Z:\working code\mrms_eval\grizzly.py:24: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  gage = [gage[i].drop(gage[i].columns.difference(['TimeStamp (Local)','15-minute Intensity (mm/h)','Bin Accum (mm)']), 1) for i in range(len(gage_id))]
Z:\working code\mrms_eval\usgs_other.py:10: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  gage = [gage[i].drop(gage[i].columns.difference(['time','precip_mm_diff','intensity_15m']), 1) for i in range(len(gage))]


In [3]:
lat = [coord[i][0] for i in range(len(coord))]
lon = [coord[i][1] for i in range(len(coord))]

In [4]:
mrms_rate_folder = os.path.join(parentDir,"MRMS","2min_rate_cat_month_CO","eval")
months = ['may','jun','jul','aug','sep']
years = [2021,2022]

filenames_rate = [mrms_rate_folder+'\\'+i+'_'+str(j)+'_rate_CO.grib2' for i in months for j in years]

mrms_rate = xr.open_mfdataset(filenames_rate,engine = "cfgrib",chunks={'time': '500MB'})
mrms_rate = mrms_rate.where(mrms_rate.longitude<=256,drop=True)

mrms_rate = mrms_rate.sel(longitude=lon,latitude=lat,method='nearest',drop=True)
mrms_rate = mrms_rate.drop_duplicates(dim=['latitude','longitude'])

# open storm_ids
storm_folder = os.path.join(parentDir,"CO_storm_output")

filenames_storm = [i+'_'+str(j)+'storm_id.nc' for i in months for j in years]

storm_id = []
# add float to make storm_ids unique for each month 
unique = np.arange(0,0.1,0.01)

for i in range(len(filenames_storm)):
    storm = xr.open_dataset(storm_folder+'\\'+filenames_storm[i],chunks={'time': '500MB'})
    storm['storm_id'] = storm.storm_id.astype('float64')
    storm = storm+unique[i]
    storm_id.append(storm)

storm_id = xr.concat(storm_id,dim='time')
storm_id = storm_id.sortby('time')

storm_id = storm_id.sel(longitude=lon,latitude=lat,method='nearest',drop=True)
storm_id = storm_id.drop_duplicates(dim=['latitude','longitude'])

Ignoring index file 'Z:\\MRMS\\2min_rate_cat_month_CO\\eval\\may_2021_rate_CO.grib2.923a8.idx' incompatible with GRIB file
Ignoring index file 'Z:\\MRMS\\2min_rate_cat_month_CO\\eval\\may_2022_rate_CO.grib2.923a8.idx' incompatible with GRIB file
Ignoring index file 'Z:\\MRMS\\2min_rate_cat_month_CO\\eval\\jun_2021_rate_CO.grib2.923a8.idx' incompatible with GRIB file
Ignoring index file 'Z:\\MRMS\\2min_rate_cat_month_CO\\eval\\jun_2022_rate_CO.grib2.923a8.idx' incompatible with GRIB file
Ignoring index file 'Z:\\MRMS\\2min_rate_cat_month_CO\\eval\\jul_2021_rate_CO.grib2.923a8.idx' incompatible with GRIB file
Ignoring index file 'Z:\\MRMS\\2min_rate_cat_month_CO\\eval\\jul_2022_rate_CO.grib2.923a8.idx' incompatible with GRIB file
Can't read index file 'Z:\\MRMS\\2min_rate_cat_month_CO\\eval\\aug_2021_rate_CO.grib2.923a8.idx'
Traceback (most recent call last):
  File "C:\Users\whitep\AppData\Local\miniconda3\envs\radar\lib\site-packages\cfgrib\messages.py", line 547, in from_indexpath_or_

In [5]:
mrms_accum = mrms_rate*(2/60)

# change time to MST
mrms_accum = time_change(mrms_accum)
storm_id = time_change(storm_id)

m = mrms_accum
s = storm_id

m = m.assign({"storm_id": s.storm_id})

In [6]:
m = m.chunk({'time': '10MB'})

In [7]:
m = m.sortby(m.latitude)
m = m.sortby(m.longitude)

C:\Users\whitep\AppData\Local\miniconda3\envs\radar\lib\site-packages\xarray\core\indexing.py:1379: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]
C:\Users\whitep\AppData\Local\miniconda3\envs\radar\lib\site-packages\xarray\core\indexing.py:1379: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return se

In [8]:
m = m.where(m.storm_id>=1,drop=True)

In [9]:
name = 'mrms_atgage_stormsonly_radaronly.nc'
path = parentDir+name
m.to_netcdf(path=path)